# Validate & Deploy – PHP Lambda API (Cloud-Ready)

This guide verifies functional integrity and deploys the `php-function.zip` package to AWS Lambda using the CLI. It's built to support audit logs, future onboarding, and minimal-friction handoffs.

---

## 🛠 Prerequisites

Ensure the following are configured:

- AWS CLI authenticated (`aws sts get-caller-identity`)
- IAM role has Lambda & S3 write access
- Bootstrap + `lambda.php` + `routes/` are zipped and ready

---

## 💼 Packaging & Upload

### Create Deployment Zip

```bash
cd D:\AWS\php-api
create-zip.bat
```

### Deploy New Function

```bash
aws lambda create-function \
  --function-name php-api-dispatcher \
  --runtime provided.al2 \
  --role arn:aws:iam::280762879731:role/lambda-execution-role \
  --handler bootstrap \
  --zip-file fileb://php-function.zip \
  --timeout 30 \
  --memory-size 256
```

### Update Existing Function

```bash
aws lambda update-function-code \
  --function-name php-api-dispatcher \
  --zip-file fileb://php-function.zip
```

---

## 🧪 Validation Tests

### Test Core Endpoints

```bash
# Health check
aws lambda invoke --function-name php-api-dispatcher \
  --payload '{"path":"/api/health"}' health-response.json
cat health-response.json

# Info endpoint
aws lambda invoke --function-name php-api-dispatcher \
  --payload '{"path":"/api/info"}' info-response.json
cat info-response.json

# Route listing
aws lambda invoke --function-name php-api-dispatcher \
  --payload '{"path":"/routes"}' routes-response.json
cat routes-response.json
```

### Create Function URL

```bash
aws lambda create-function-url-config \
  --function-name php-api-dispatcher \
  --auth-type NONE
```

### Test Function URL

```bash
# Replace with your actual Function URL
curl https://[function-url]/api/health
curl https://[function-url]/api/info
curl https://[function-url]/routes
```

---

## 📊 Monitoring & Logs

### Check Function Status

```bash
aws lambda get-function --function-name php-api-dispatcher \
  --query 'Configuration.{State:State,LastUpdateStatus:LastUpdateStatus}'
```

### View Logs

```bash
aws logs get-log-events \
  --log-group-name "/aws/lambda/php-api-dispatcher" \
  --log-stream-name "$(aws logs describe-log-streams \
    --log-group-name '/aws/lambda/php-api-dispatcher' \
    --order-by LastEventTime --descending --limit 1 \
    --query 'logStreams[0].logStreamName' --output text)"
```

---

## 🔄 CloudFront Integration

### Update CloudFront Origin

```bash
# Get current distribution config
aws cloudfront get-distribution-config --id [distribution-id] > current-config.json

# Update origin domain to Function URL
# Then update distribution with new config
aws cloudfront update-distribution \
  --id [distribution-id] \
  --distribution-config file://updated-config.json \
  --if-match [etag]
```

---

## ✅ Success Criteria

- [ ] Function deploys without errors
- [ ] `/api/health` returns 200 with PHP version
- [ ] `/api/info` returns runtime metadata
- [ ] `/routes` lists available endpoints
- [ ] Function URL accessible via HTTPS
- [ ] CloudFront routes to Function URL
- [ ] Logs show successful invocations

---

## 🚨 Troubleshooting

### Common Issues

- **Bootstrap not executable**: Ensure bootstrap has proper shebang
- **PHP binary not found**: Verify `/opt/php/bin/php` path or add PHP layer
- **Runtime timeout**: Increase timeout or optimize code
- **Permission denied**: Check IAM role has Lambda execution permissions

### Debug Commands

```bash
# Check function configuration
aws lambda get-function-configuration --function-name php-api-dispatcher

# Test with verbose logging
aws lambda invoke --function-name php-api-dispatcher \
  --payload '{"path":"/api/health"}' \
  --log-type Tail response.json
```